<a href="https://colab.research.google.com/github/cateto/python4NLP/blob/main/kobert/load_model_success.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
# 변경: 최신 버전으로 설치하면 "Input: must be Tensor, not str" 라는 에러 발생
!pip install transformers==3
!pip install torch

     |████████████████████████████████| 46.9 MB 43 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 5.0 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595739 sha256=c8aee0c5fee230b35edfc0e56b86e5b6a72b662a22a32b7d79fd35b9de41e5ee
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 4.7 MB/s 
     |████████████████████████████████| 754 kB 4.9 MB/s 
     |████████████████████████████████| 895 kB 42.5 MB/s 
     |████████████████████████████████| 3.0 MB 55.0 MB/s 


In [6]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-r7ffyeqz
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-r7ffyeqz
  Created wheel for kobert: filename=kobert-0.1.2-py3-none-any.whl size=12770 sha256=6470f128f625386780d89de2858b62ef8a0cbfc26b64b6d1c865a3acfa293dad
  Stored in directory: /tmp/pip-ephem-wheel-cache-1ef43_4g/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd051c0
Successfully built kobert


In [7]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [8]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768, # bert의 hidden layer 크기?
                #  num_classes=2,
                 num_classes=9, # 분류 class 크기
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes) #nn.Linear
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [16]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [22]:
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
using cached model


In [23]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [24]:
## Setting parameters
max_len = 128 # max_length 512, but Considering Computing Resource => set 163
batch_size = 16 # GPU CUDA problem
warmup_ratio = 0.1
num_epochs = 4
max_grad_norm = 1
log_interval = 200
learning_rate = 1e-5

In [15]:
import torch
device = torch.device('cpu')
model = torch.load('/content/drive/MyDrive/rsn_nlp_project/20210802model_95.pt', map_location=device)

In [45]:
# 테스트 문장 예측

test_sentence = '''“모기 한 마리 왱 하고 귓가에 들려오면 / 기가 질려 속이 타고 간담이 서늘하다. (…) 싸워봐야 소용없고 잠만 공연히 못 자기에 / 여름밤이 지루하기 1년과 맞먹는다.”
다산 정약용은 모기를 향한 증오를 ‘증문(憎蚊)’이라는 시로 표현했다. 모기는 귀찮을 뿐 아니라 위험하기까지 하다. 사람의 피를 빨아들이는 과정에서 말라리아, 황열, 뎅기열, 뇌염 등의 질병을 옮기기 때문이다. 모기가 매개하는 질환으로 매년 전 세계에서 최소 72만명이 사망한다. 국내에서도 최근 5년간 일본뇌염 환자 95명이 발생했고, 이 중 12명이 사망했다.

여름철 대표 불청객 모기가 올해는 예년보다 덜 보인다. 2일 서울시에 따르면 서울 시내 디지털모기측정기(DMS) 50개소에서 채집된 모기는 지난달 14~24일 기준으로 총 3만3906마리다. 일평균 약 3082마리가 포획된 셈이다.

다른 해와 비교하면 확연히 적은 숫자다. 서울시가 집계를 시작한 2015년 이후로 가장 낮다. 지난해 같은 기간(3만9032마리)과 비교하면 13.1% 줄었다. 수치가 가장 높았던 2016년(6만9433마리) 대비로는 무려 51.2% 감소했다.

지난달 들어 불볕더위가 유례없이 지속한 데 따른 영향으로 풀이된다. 기상청 기상자료개방포털에 따르면 서울 최고기온은 7월 12일부터 31일까지 20일 연속으로 30도를 넘어섰다. ‘역대급’ 폭염으로 기록된 2018년 7월(38.3도)보다 최고기온은 1.8도 낮았지만, 평균기온(28.1도)은 오히려 0.3도 높았다.'''
test_label = '5'
temp_set = []
temp_set.append(test_sentence)
temp_set.append(test_label)
test_set = []
test_set.append(temp_set)

In [46]:
test_set = BERTDataset(test_set, 0, 1, tok, max_len, True, False)
test_input = torch.utils.data.DataLoader(test_set, batch_size=1, num_workers=9)

for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_input)):
  token_ids = token_ids.long().to(device)
  segment_ids = segment_ids.long().to(device)
  valid_length= valid_length
  out = model(token_ids, valid_length, segment_ids)
  print(out)


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 9 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


tensor([[-1.1078, -1.4059, -0.2804, -0.4595,  2.5718,  4.7971, -0.6574, -1.5570,
         -1.5634]], grad_fn=<AddmmBackward>)

